# Project Overview

This project involves the discovery of promising materials for organic solar cells. The focus is on the HOMO-LUMO gap, a measure of molecular efficiency for solar energy use. The larger this gap, the more efficient the molecule is for solar cells. The challenge is to use machine learning to predict this gap from a molecule description, which is a more efficient approach than traditional density functional theory.

The data provided includes a small data set (100 molecules with associated HOMO-LUMO gaps) and a larger, related data set (50,000 molecules with LUMO energy levels). Despite the second data set's labels not being directly the desired prediction target, it is considered valuable due to the probable correlation between molecule features predictive of LUMO energy and the HOMO-LUMO gap.

This approach is referred to as transfer learning, where features learned on one task are used for another related task. The project also suggests using unsupervised representation learning methods, like autoencoders, to extract useful feature representations from the larger dataset. Knowledge should be "transferred" to utilize the additional label information in the larger dataset. This could involve, for example, reusing the last layer of a neural network trained to predict LUMO energy.

The goal is to submit predictions of the HOMO-LUMO gap for a third set of 10,000 unseen molecule descriptions.

# Methodology

The code employs a hybrid approach combining an autoencoder with regression models for a predictive task. This approach uses semi-supervised learning, beneficial given the extensive data requirements of deep learning models. Instead of using pre-provided fingerprints, the RDKit library is used to extract MACCS keys and ECFP (2000 bits) from the smiles of each molecule, thereby expanding the dataset. This methodology is inspired by the paper "Improvement of Prediction Performance With Conjoint Molecular Fingerprint in Deep Learning".

The autoencoder in this code doesn't compress the input but seeks to learn valuable feature transformation. This feature transformation is then used in conjunction with a predictor model trained on the encoder's output to predict the target variable, LUMO. This dual-task structure encourages the encoder to learn features beneficial for both reconstructing the original data and predicting the target variable. 


The reason the encoder doesn't compress the input in this case is because we're more interested in feature transformation than dimensionality reduction. By not compressing the input, the encoder can learn a more complex and potentially useful representation of the data, which can then be used by the predictor model to make more accurate predictions. This approach is particularly useful in cheminformatics, where the input data (molecular fingerprints) is already a compressed representation of the original data (molecular structures), and further compression could result in loss of important information.

After training the autoencoder-predictor on the larger pretraining dataset, the encoder, now acting as a feature extractor, is used to transform the actual training data into a latent representation. This representation captures significant patterns relevant to a similar task. Then, two regression models are trained on these encoded features, exploiting the learned patterns for the prediction task.


## Set Up

This section is dedicated to setting up the necessary environment for the project. This includes importing the required libraries and defining helper functions.

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from rdkit import Chem, DataStructs
from rdkit.Chem import MACCSkeys, AllChem
from tensorflow.keras.models import Model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Dense,Input, Dropout, BatchNormalization,LeakyReLU

### Helper Functions

Two helper functions are defined here:

- `smiles_to_fp`: This function converts a molecule's SMILES representation to a fingerprint. It uses the RDKit library to generate Extended-Connectivity Fingerprints (ECFP) and MACCS keys for the molecule. These fingerprints are then concatenated to form the final feature set.

- `rmse`: This function calculates the root mean square error (RMSE) between two arrays. It is used for model evaluation.

In [4]:
# Function to convert smiles to fingerprint
def smiles_to_fp(smiles, radius=2, n_bits=2000):
    # Convert smiles to molecule
    mol = Chem.MolFromSmiles(smiles)
    # Generate ECFP
    ecfp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
    # Generate MACCS keys
    maccs = MACCSkeys.GenMACCSKeys(mol)
    # Convert ECFP and MACCS keys to numpy arrays
    arr_ecfp = np.zeros((0,), dtype=int)
    arr_maccs = np.zeros((0,), dtype=int)
    DataStructs.ConvertToNumpyArray(ecfp, arr_ecfp)
    DataStructs.ConvertToNumpyArray(maccs, arr_maccs)
    # Concatenate ECFP and MACCS keys
    conjoint_fp = np.concatenate([arr_ecfp, arr_maccs])
    return conjoint_fp

In [5]:
# Function to calculate root mean square error
def rmse(x1, x2):
    return np.sqrt(mean_squared_error(x1, x2))

## Data Loading and Preprocessing

In this section, we load the data and perform some preprocessing steps. The data consists of two datasets:

- A small dataset of 100 molecules with associated HOMO-LUMO gaps.
- A larger dataset of 50,000 molecules with LUMO energy levels.

The small dataset is used for the main task of predicting the HOMO-LUMO gap, while the larger dataset is used for pretraining the model in a transfer learning approach.

### Data Preprocessing

The data preprocessing step involves converting the SMILES strings into molecular fingerprints. The RDKit library is used to convert the SMILES strings into molecule objects, and then the MACCS keys and ECFP fingerprints are generated from these molecule objects.

The MACCS keys are a set of 167 substructure key descriptors, and the ECFP fingerprints are a type of circular fingerprint that is based on the Morgan algorithm. These fingerprints are binary vectors that represent the presence or absence of certain molecular substructures, and they are a common way of encoding molecular structure information for machine learning models.

The fingerprints are then concatenated together to form a single feature vector for each molecule. This results in a feature vector of length 2167 for each molecule.

The target variable for the small dataset, the HOMO-LUMO gap, is also extracted.

In [ ]:
def convert_smiles_to_fingerprints(smiles_array):
    return np.array([smiles_to_fp(s) for s in smiles_array])

In [ ]:
def load_and_preprocess_data(file_name):
    # Load data
    df = pd.read_csv(file_name, index_col="Id", compression='zip')
    # Extract SMILES strings
    smiles = df['smiles'].to_numpy()
    # Convert SMILES strings to fingerprints
    fingerprints = convert_smiles_to_fingerprints(smiles)
    return fingerprints

In [ ]:
# Load and preprocess pretraining data
pretrain_x_conjoint = load_and_preprocess_data("pretrain_features.csv.zip")
pretrain_y = pd.read_csv("pretrain_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
print('Pretraining data loaded')

In [ ]:
# Load and preprocess training data
train_x_conjoint = load_and_preprocess_data("train_features.csv.zip")
train_y = pd.read_csv("train_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
print('Training data loaded')

In [ ]:
# Load and preprocess test data
test_df = pd.read_csv("test_features.csv.zip", index_col="Id", compression='zip')
test_x = test_df['smiles'].to_numpy()
test_x_conjoint = convert_smiles_to_fingerprints(test_x)
print('Loaded test data')

## Model Creation

The model for this task is a combination of an autoencoder and a predictor. The autoencoder is a type of neural network that is used to learn efficient data codings in an unsupervised manner. It consists of an encoder, which compresses the input data into a latent-space representation, and a decoder, which reconstructs the input data from the latent space representation.

In this case, the autoencoder is not used to reduce the dimensionality of the input data (which is a common use case for autoencoders), but rather to learn a transformation of the input data that makes it easier for the predictor to predict the target variable (the HOMO-LUMO gap). This is achieved by training the autoencoder and the predictor simultaneously, with the predictor taking the output of the encoder as input.

The reason for not compressing the input data in the encoder is that we are more interested in transforming the data into a representation that is useful for the prediction task, rather than reducing the dimensionality of the data. By allowing the encoder to learn a higher-dimensional representation, we give it more flexibility to learn a complex transformation that captures the important features in the data for predicting the HOMO-LUMO gap.

This approach is inspired by the concept of representation learning, which is a set of techniques that allows a system to automatically discover the representations needed for data analysis or task performance. In the context of deep learning, representation learning is often used to learn complex data transformations that can improve the performance of the model on the task at hand.

The predictor is a simple feed-forward neural network that takes the output of the encoder as input and predicts the HOMO-LUMO gap. The use of a neural network for the predictor allows us to capture complex, non-linear relationships between the transformed input data and the target variable.

### Autoencoder

In this project, the autoencoder is used to learn a valuable feature transformation from the input data. The learned features are then used for predicting the target variable, LUMO.

#### **Design Choices**


- **Linear Activation for Encoder Output**: The linear activation function was used for the output of the encoder to allow it to output a wide range of values, not limited to a specific range or distribution. This is appropriate for the task of learning a feature transformation, where we want to preserve as much information as possible about the input data.

- **Softmax Activation for Decoder Output Layer**: The softmax activation function was used for the output layer of the decoder to ensure that the output is a probability distribution, i.e., that the output values are non-negative and sum to 1. This is appropriate for the task of reconstructing the input data, which in this case is a binary fingerprint.

- **Leaky ReLU Activation Function**: The Leaky ReLU activation function was chosen for its ability to mitigate the vanishing gradients problem, which can occur when training deep neural networks. By allowing a small, positive gradient when the unit is not active, Leaky ReLU helps to ensure that neurons never die, i.e., go into a state where they could possibly stop learning.




In [11]:
# Function to create encoder model
def create_encoder(input_dim=2167):
    inputs = Input(shape=(input_dim,))
    x = Dense(2167)(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = Dropout(0.5)(x)
    x = Dense(2500)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = Dropout(0.5)(x)
    x = Dense(2167,'linear')(x)
    return Model(inputs, x, name='encoder')

In [12]:
# Function to create decoder model
def create_decoder(input_dim=2167):
    inputs = Input(shape=(input_dim,))
    x = Dense(2000)(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = Dropout(0.5)(x)
    decoded = Dense(2167, activation='softmax')(x)
    return Model(inputs, decoded, name='decoder')

### Predictor Model

The predictor model is a simple neural network that is trained on the output of the encoder to predict the target variable, LUMO. This model is trained simultaneously with the autoencoder, encouraging the encoder to learn features beneficial for both reconstructing the original data and predicting the target variable.

In [13]:
# Function to create predictor model
def create_predictor(input_dim=2167):
    inputs = Input(shape=(input_dim,))
    x = Dense(256)(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = Dropout(0.5)(x)
    output = Dense(1, 'linear')(x)
    return Model(inputs, output, name='predictor')

### Combined Model

The combined model is a combination of the autoencoder and the predictor model. It is trained on the larger pretraining dataset to learn valuable feature transformations and predict the LUMO energy levels. After training, the encoder part of the model is used as a feature extractor to transform the actual training data into a latent representation. This representation captures significant patterns relevant to a similar task.

In [14]:
# Function to create combined model
def create_combined_model(input_dim=2167):
    # Create encoder, decoder, and predictor models
    encoder = create_encoder()
    decoder = create_decoder()
    predictor = create_predictor()
    # Define inputs
    inputs = Input(shape=(input_dim,))
    # Pass inputs through encoder
    encoded = encoder(inputs)
    # Pass encoded inputs through decoder and predictor
    decoded = decoder(encoded)
    predicted = predictor(encoded)
    # Define model
    model = Model(inputs, [decoded, predicted], name='combined')
    # Compile model
    model.compile(loss={'decoder':'binary_crossentropy', 'predictor':'mse'}, optimizer=SGD(momentum=0.6), metrics={'predictor': RootMeanSquaredError(name = 'rmse')})
    return model, encoder

In [15]:
# Create combined model and encoder
model, encoder = create_combined_model()
print('Model and encoder created')

#### **Model Summary**

In [16]:
# Print model summary
model.summary()

## Model Training

In this section, we train the models. The combined model (autoencoder + predictor) is first trained on the larger pretraining dataset. After training, the encoder part of the model is used to transform the actual training data into a latent representation. Two regression models (Linear Regression and XGBoost) are then trained on these encoded features.

#### **Callbacks**

In [23]:
# Set up callbacks
early_stopping = EarlyStopping(monitor='val_predictor_rmse', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_predictor_rmse', factor=0.1, patience=5)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_predictor_rmse', save_best_only=True, save_weights_only=True)
callbacks = [early_stopping, reduce_lr, model_checkpoint]

#### **Fit**

In [24]:
# Fit the model
history = model.fit(pretrain_x_conjoint, [pretrain_x_conjoint, pretrain_y], validation_split=0.1, epochs=100, batch_size=32, callbacks=callbacks)

## **Regression Model**

Basic regression models (Linear Regression and XGBoost) were chosen for the final task due to their simplicity, interpretability, and strong performance on a wide range of problems. While deep learning models can achieve high performance, they also require large amounts of data and computational resources, and can be more difficult to interpret.

### **Extract features**

In [25]:
# Extract features from training data using the encoder
encoded_train_x = encoder.predict(train_x_conjoint, verbose=0)
print('Features extracted')

#### **Split validation data**

In [26]:
# Split validation data
x, x_val, y, y_val = train_test_split(encoded_train_x, train_y, test_size=0.1, random_state=42)
print('Validation data split')

## **Fit regressors**

In [27]:
# Fit linear regression model
lr = LinearRegression().fit(x, y)
print('Linear regression model fitted')

In [28]:
# Fit XGBoost regression model
xgb = XGBRegressor(tree_method='gpu_hist').fit(x, y)
print('XGBoost regression model fitted')

### **Test models**

In [40]:
# Test models
lr_rmse = rmse(y_val, lr.predict(x_val))
xgb_rmse = rmse(y_val, xgb.predict(x_val))
print(f'Linear Regression RMSE: {lr_rmse}')
print(f'XGBoost RMSE: {xgb_rmse}')

#### **Select best**

In [30]:
# Select the best model
if lr_rmse < xgb_rmse:
    best_model = lr
    print('Linear Regression model selected')
else:
    best_model = xgb
    print('XGBoost model selected')

## **Predict and save results**

### **Predict**

In [41]:
# Make predictions using the best model
encoded_test_x = encoder.predict(test_x_conjoint, verbose=0)
predictions = best_model.predict(encoded_test_x)
print('Predictions made')

### **Save**

In [43]:
# Save predictions
df_pred = pd.DataFrame({"y": predictions}, index=test_df.index)
df_pred.to_csv("results.csv", index_label="Id")
print("Predictions saved")

## Conclusion

In this project, we tackled the challenge of predicting the HOMO-LUMO gap of molecules for the application in organic solar cells. We used a hybrid approach combining an autoencoder with regression models to leverage a larger dataset with related but not directly applicable labels. This approach, known as transfer learning, allowed us to extract valuable feature representations from the larger dataset and apply them to our smaller, target dataset.

We used the RDKit library to extract MACCS keys and ECFP from the smiles of each molecule, expanding our dataset and providing a more detailed representation of each molecule. Our autoencoder was designed not to compress the input but to learn valuable feature transformations, which were then used in conjunction with a predictor model to predict the target variable, LUMO.

The results showed that our approach was able to effectively predict the HOMO-LUMO gap, demonstrating the potential of machine learning techniques in the field of materials science and specifically in the design of organic solar cells.

## Future Work

While the results of this project are promising, there are several directions for future work to improve and expand upon what we've done:

- **Model Exploration**: We could explore other types of models or architectures, such as different types of autoencoders or other types of neural networks such as Graph Neural Networks (GNNs).

- **Feature Engineering**: We could explore other ways of representing the molecules, such as different types of fingerprints or other molecular descriptors. We could also look into more advanced feature selection techniques to identify the most informative features.

- **Data Augmentation**: We could look into techniques for data augmentation, such as generating new molecules or modifying existing ones, to increase the size of our training dataset.

- **Hyperparameter Tuning**: We could perform more extensive hyperparameter tuning to optimize the performance of our models. This could include tuning the architecture of the autoencoder and the predictor model, as well as the training parameters.


# References

1. [Improvement of Prediction Performance With Conjoint Molecular Fingerprint in Deep Learning](https://www.frontiersin.org/articles/10.3389/fchem.2020.00606/full)
2. [A Novel Molecular Representation Learning for Molecular Property Prediction with a Multiple SMILES-Based Augmentation](https://www.mdpi.com/2218-273X/11/8/1220)